### Spark session configuration
This cell sets Spark session settings to enable _Verti-Parquet_ and _Optimize on Write_. More details about _Verti-Parquet_ and _Optimize on Write_ in tutorial document.

In [ ]:
# Copyright (c) Microsoft Corporation.
# Licensed under the MIT License.

spark.conf.set("spark.sql.parquet.vorder.enabled", "true")
spark.conf.set("spark.microsoft.delta.optimizeWrite.enabled", "true")
spark.conf.set("spark.microsoft.delta.optimizeWrite.binSize", "1073741824")

### Fact - Sale

This cell reads raw data from the _Files_ section of the lakehouse, adds additional columns for different date parts and the same information is being used to create partitioned fact delta table.

In [ ]:
from pyspark.sql.functions import col, year, month, quarter

table_name = 'ExternalStorage_Fact_Sales'

df = spark.read.format("parquet").load('Files/ExternalSource_Sales')
df = df.withColumn('Year', year(col("InvoiceDateKey")))
df = df.withColumn('Quarter', quarter(col("InvoiceDateKey")))
df = df.withColumn('Month', month(col("InvoiceDateKey")))

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import DateType
import random
import datetime


# Function to generate random date between 2005 and 2014
def random_date():
    start_date = datetime.date(2005, 1, 1)
    end_date = datetime.date(2014, 12, 31)
    delta = end_date - start_date
    random_days = random.randint(0, delta.days)
    return start_date + datetime.timedelta(days=random_days)


# Register UDF
random_date_udf = udf(random_date, DateType())


# Add new column with random dates
df = df.withColumn("OrderDate", random_date_udf())

# Show DataFrame
df.show()



In [ ]:
#Write the dataframe
df.write.mode("append").format("delta").partitionBy("Year","Quarter").save("Tables/" + table_name)